AIM: generates the SSH corrected from the the global mean sea level at each time step and for each simulation

In [1]:
## standart libraries

import os,sys
import numpy as np

# xarray
import xarray as xr

import multiprocessing.popen_spawn_posix
from distributed import Client

# from dask.distributed import Client
c = Client()
# c = Client(n_workers=6, threads_per_worker=1, memory_limit="8 GiB",local_directory="/gpfsscratch/rech/cli/uor98hu/dask_tmp/")
# import lib_SLXtools_IMHOTEP_2021 as slx
c

Client Scheduler: tcp://127.0.0.1:37481 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 10 Memory: 20.97 GB


# Loading data

In [2]:
# ------ mesh for constructing the weight for the computation of the global mean sea level
chunk_size2 = {"x":133,"y":320}
diri="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
mesh_hgr = xr.open_dataset(diri+'mesh_hgr.nc',chunks=chunk_size2).squeeze()
tmask = mesh_hgr.tmask

e1t = mesh_hgr.e1t.fillna(0)
e2t = mesh_hgr.e2t.fillna(0)

nav_lon =  mesh_hgr.nav_lon
nav_lat =  mesh_hgr.nav_lat

# constructing the weight
bt = (e1t * e2t).load()# area of each cell
# getting the weights to be applied to every grid cell
# masking the land cells
oceanArea = (bt*tmask[0])  #tmask : 1 on ocean, 0 on land
totalOceanArea = oceanArea.sum()
Weight = (oceanArea/totalOceanArea).compute()


In [3]:
# loading the SSH from any run 
## loading data with for a given experience

#-------- Read data
prefix = "eORCA025.L75-IMHOTEP"
varna = 'sossheig'

fo="1m" # output frequency

# input directory on my work
chunk_size = {"x":133,"y":320}

for nexp in ["S", "GAI", "AI"]:

    print(nexp)
    diridatref="/gpfsstore/rech/cli/rcli002/eORCA025.L75/"+prefix+"."+nexp+"-S/"+fo+"/"
    data = xr.open_mfdataset(diridatref+'????/*_gridTsurf.nc', chunks = chunk_size)[varna].fillna(0).squeeze()

    # calculating the GMSL
    GMSL = (data*Weight).sum(('x','y')).compute()
    print('GMSL done')
    # correction of the GMSL
    dataCorrected = data - GMSL
    print('correction done')
        
    #convert into dataset
    sshEds = xr.Dataset(
        data_vars=dict(sossheig=(["time_counter", "y", "x"], dataCorrected.data),
                      GMSL = (["time_counter"], GMSL.data) ),
        coords=dict(time_counter=data.time_counter.values),
        attrs=dict(
            description=" sea level heigh corrected from the GMSL",
            units="m",
            interval_write = fo))

    namo = prefix+'.'+nexp+'_y1980_2018.'+fo+'_sossheigCorrected.nc'
    print(namo)
    sshEds.to_netcdf(path='/gpfswork/rech/cli/uor98hu/MYDATA/SSH/'+nexp+'/'+namo, mode='w')



S
GMSL done
correction done
eORCA025.L75-IMHOTEP.S_y1980_2018.1m_sossheigCorrected.nc


PermissionError: [Errno 13] Permission denied: b'/gpfswork/rech/cli/uor98hu/MYDATA/SSH/S/eORCA025.L75-IMHOTEP.S_y1980_2018.1m_sossheigCorrected.nc'

In [5]:
sshEds.to_netcdf(path='/gpfswork/rech/cli/uor98hu/MYDATA/SSH/'+nexp+'/'+namo, mode='w')